In [1]:
from Core.Preprocess import *
from Core.CryptoCompareAPI import *
from Core.config_reader import config_reader

In [84]:
import pandas as pd
import calendar

## Download Data

In [3]:
reader = config_reader('/Users/admin/PycharmProjects/Market_Prediction_Using_News_and_Indicators/config.yml')

crypto_currency_list = reader.read_config("CRYPTO_CURRENCIES_LIST")
exchange = reader.read_config("EXCHANGE")
currency = reader.read_config("CURRENCY")

In [74]:
df_crypto_list = []

In [75]:
 for crypto in crypto_currency_list:
    raw_data = get_historical_price_hour_from(
        coin=crypto,
        currency=currency,
        exchange=exchange,
        toTs=int(calendar.timegm(datetime.datetime(2022, 5, 1, 0, 0, 0).timetuple())),
        fromTs=int(calendar.timegm(datetime.datetime(2022, 2, 1, 0, 0, 0).timetuple()))
    )
    data = Preprocess.get_from_crypto_compare_hourly_price_data_to_pandas(raw_data)
    df_crypto_list.append(data)

## Preprocessing

In [76]:
df_crypto_list[0]

,close,high,low,open,volumefrom,volumeto,conversionType,conversionSymbol,datetimes
time,,,,,,,,,
2022-02-01 00:00:00,38342.36,38627.35,38276.43,38466.90,1058.43,4.064914e+07,force_direct,,2022-02-01
2022-02-01 01:00:00,38238.19,38407.00,38175.86,38342.36,777.86,2.978374e+07,force_direct,,2022-02-01
2022-02-01 02:00:00,38480.90,38506.18,38196.61,38238.19,766.84,2.941555e+07,force_direct,,2022-02-01
2022-02-01 03:00:00,38596.82,38649.90,38384.99,38480.90,946.46,3.646598e+07,force_direct,,2022-02-01
2022-02-01 04:00:00,38556.82,38644.57,38470.37,38596.82,758.93,2.926184e+07,force_direct,,2022-02-01
...,...,...,...,...,...,...,...,...,...
2022-04-30 20:00:00,38326.45,38466.83,38282.08,38466.83,676.71,2.596374e+07,force_direct,,2022-04-30
2022-04-30 21:00:00,38301.14,38370.84,38248.66,38326.45,591.08,2.264638e+07,force_direct,,2022-04-30
2022-04-30 22:00:00,38250.44,38370.00,38160.00,38301.14,1376.35,5.267822e+07,force_direct,,2022-04-30


In [77]:
for i in range(len(df_crypto_list)):
    Preprocess.rename_columns(df_crypto_list[i])

In [78]:
df_crypto_list[0]

,Close,High,Low,Open,Volume
time,,,,,
2022-02-01 00:00:00,38342.36,38627.35,38276.43,38466.90,4.064914e+07
2022-02-01 01:00:00,38238.19,38407.00,38175.86,38342.36,2.978374e+07
2022-02-01 02:00:00,38480.90,38506.18,38196.61,38238.19,2.941555e+07
2022-02-01 03:00:00,38596.82,38649.90,38384.99,38480.90,3.646598e+07
2022-02-01 04:00:00,38556.82,38644.57,38470.37,38596.82,2.926184e+07
...,...,...,...,...,...
2022-04-30 20:00:00,38326.45,38466.83,38282.08,38466.83,2.596374e+07
2022-04-30 21:00:00,38301.14,38370.84,38248.66,38326.45,2.264638e+07
2022-04-30 22:00:00,38250.44,38370.00,38160.00,38301.14,5.267822e+07


In [79]:
df_crypto_list = list(map(Preprocess.compute_all_indicators, df_crypto_list))

In [80]:
for i in range(len(df_crypto_list)):
    df_crypto_list[i].drop(columns=["Close", "High", "Low", "Open", "Volume"], inplace=True)
    df_crypto_list[i] = df_crypto_list[i].iloc[90:]

In [83]:
df_crypto_list[0]

,UpperBB,LowerBB,DEMA,EMA,HT_TRENDLINE,KAMA,MA,MIDPOINT,MIDPRICE,SAR,...,STOCHRSI FASTD,TRIX,ULTOSC,WILLR,AD,ADOSC,OBV,TRANGE,ATR,NATR
time,,,,,,,,,,,,,,,,,,,,,
2022-02-04 18:00:00,40353.302329,35918.871671,38724.151122,38100.630020,37434.617702,38402.621527,37660.167667,38970.930,38961.010,36250.00,...,98.476603,-0.011691,64.119127,-5.665491,4.163835e+08,2.232880e+08,8.408365e+07,383.22,520.992874,0.003181
2022-02-04 19:00:00,40769.482460,35844.321540,39011.992214,38270.131955,37518.339136,38739.630392,37798.262000,39074.655,39083.600,36250.00,...,98.476603,-0.002145,64.504258,-4.156630,4.559270e+08,2.169142e+08,1.792901e+08,510.01,520.208383,0.003136
2022-02-04 20:00:00,41079.282361,35875.268639,39253.562275,38419.671829,37598.399669,38997.767312,37934.338333,39074.655,39087.580,36250.00,...,65.143270,0.007877,67.242522,-8.260913,4.157808e+08,1.818135e+08,7.511321e+07,428.71,513.672784,0.003118
2022-02-04 21:00:00,41343.475235,35956.630765,39475.035920,38561.896227,37675.517391,39207.938299,38058.141667,39074.655,39087.580,36250.00,...,35.991913,0.018190,67.727990,-7.255283,4.722276e+08,1.688860e+08,1.361546e+08,205.66,491.671871,0.002979
2022-02-04 22:00:00,41573.542988,36058.112012,39675.833623,38696.079696,37766.415297,39423.053828,38189.966333,39074.655,39087.580,36250.00,...,6.690602,0.028623,65.822548,-6.765959,5.104049e+08,1.603915e+08,1.825797e+08,284.14,476.848166,0.002887
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-04-30 20:00:00,38817.330196,38245.105804,38340.412243,38619.268306,38543.876250,38734.695653,38554.590667,38454.305,38451.095,34322.28,...,73.273558,-0.039017,44.091768,-71.122333,1.662901e+09,-4.149096e+07,-1.322932e+09,184.75,217.651086,0.001482
2022-04-30 21:00:00,38803.026319,38219.327681,38318.678299,38598.743899,38528.550500,38715.576556,38531.133333,38440.385,38422.380,34322.28,...,60.449300,-0.039700,49.901629,-72.760381,1.659709e+09,-3.484338e+07,-1.345579e+09,122.18,210.831723,0.001437
2022-04-30 22:00:00,38789.827932,38188.271068,38293.254365,38576.272680,38509.905800,38707.654251,38522.070333,38436.470,38407.380,34322.28,...,27.115967,-0.040374,44.428035,-81.220657,1.652405e+09,-3.128056e+07,-1.398257e+09,210.00,210.772314,0.001441


## Build Models

In [86]:
from sklearn.model_selection import train_test_split

### MLP

In [85]:
from sklearn.neural_network import MLPClassifier